<a href="https://colab.research.google.com/github/Satish-970/ATS_Checker_Job-Description/blob/main/ATS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### ATS Resume Matcher: Your Guide to a Better Resume

This tool helps you evaluate how well your resume matches a job description, providing an ATS score and suggestions for improvement. It's designed to help you pass through Applicant Tracking Systems (ATS) and get your resume seen by recruiters.

To use it, simply run all the code cells sequentially. You'll be prompted to upload your resume as a PDF or DOC file and paste the job description you're applying for.

No sign-in required - just upload your resume and get instant results!

In [ ]:
# Step 1: Install and Import Libraries

# Install necessary libraries
!pip install spacy PyPDF2 scikit-learn python-docx
!python -m spacy download en_core_web_sm

# Import libraries
import spacy
import PyPDF2
from docx import Document
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
from google.colab import files
from IPython.display import clear_output
import os

print("Setup complete!")

In [ ]:
# Step 2: Define Core Functions

# Load the spaCy model
try:
    nlp = spacy.load('en_core_web_sm')
except OSError:
    print("Downloading spaCy model...")
    !python -m spacy download en_core_web_sm
    nlp = spacy.load('en_core_web_sm')

# Function to extract text from PDF
def extract_pdf_text(pdf_path):
    """
    Extracts text from a PDF file.
    """
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ''
        for page in reader.pages:
            text += page.extract_text() + ' '
    return text.strip()

# Function to extract text from DOCX
def extract_docx_text(docx_path):
    """
    Extracts text from a DOCX file.
    """
    doc = Document(docx_path)
    text = ''
    for paragraph in doc.paragraphs:
        text += paragraph.text + ' '
    return text.strip()

# Function to extract text from uploaded resume
def extract_resume_text(file_path):
    """
    Extracts text from resume file (PDF or DOCX).
    """
    file_extension = file_path.lower().split('.')[-1]
    
    if file_extension == 'pdf':
        return extract_pdf_text(file_path)
    elif file_extension in ['docx', 'doc']:
        return extract_docx_text(file_path)
    else:
        raise ValueError("Unsupported file format. Please upload PDF or DOCX files only.")

# Function to extract keywords
def extract_keywords(text):
    """
    Extracts relevant keywords from text using NLP.
    """
    doc = nlp(text.lower())
    keywords = set()
    
    # Comprehensive list of technical and professional skills
    tech_skills = {
        'python', 'java', 'javascript', 'sql', 'html', 'css', 'react', 'angular', 
        'vue', 'node.js', 'django', 'flask', 'spring', 'hibernate', 'mongodb', 
        'postgresql', 'mysql', 'oracle', 'aws', 'azure', 'gcp', 'docker', 
        'kubernetes', 'jenkins', 'git', 'github', 'gitlab', 'agile', 'scrum', 
        'kanban', 'jira', 'confluence', 'tensorflow', 'pytorch', 'keras', 
        'pandas', 'numpy', 'scipy', 'matplotlib', 'seaborn', 'tableau', 
        'powerbi', 'excel', 'r', 'scala', 'spark', 'hadoop', 'kafka', 
        'elasticsearch', 'redis', 'graphql', 'rest', 'api', 'microservices',
        'devops', 'ci/cd', 'linux', 'windows', 'macos', 'bash', 'powershell',
        'communication', 'leadership', 'teamwork', 'management', 'analytics',
        'problem-solving', 'debugging', 'testing', 'automation'
    }
    
    # Extract nouns, proper nouns, and technical skills
    for token in doc:
        if token.pos_ in ['NOUN', 'PROPN'] and len(token.text) > 2 and token.text.isalpha():
            keywords.add(token.text)
        elif token.text in tech_skills:
            keywords.add(token.text)
    
    # Handle multi-word skills and phrases
    text_lower = text.lower()
    multi_word_skills = [
        'machine learning', 'data analysis', 'data science', 'project management',
        'artificial intelligence', 'deep learning', 'natural language processing',
        'computer vision', 'big data', 'business intelligence', 'data mining',
        'software development', 'web development', 'mobile development',
        'full stack', 'front end', 'back end', 'database management',
        'cloud computing', 'cyber security', 'network security',
        'quality assurance', 'user experience', 'user interface'
    ]
    
    for skill in multi_word_skills:
        if skill in text_lower:
            keywords.add(skill)
    
    return keywords

# Function to calculate ATS score
def calculate_ats_score(resume_text, job_description):
    """
    Calculates similarity score between resume and job description.
    """
    documents = [resume_text, job_description]
    vectorizer = TfidfVectorizer(
        stop_words='english', 
        ngram_range=(1, 2),
        max_features=1000,
        lowercase=True
    )
    tfidf_matrix = vectorizer.fit_transform(documents)
    similarity = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])[0][0]
    return similarity * 100

# Function to provide improvement suggestions
def get_suggestions(resume_keywords, job_keywords):
    """
    Provides suggestions based on missing keywords.
    """
    missing = job_keywords - resume_keywords
    if missing:
        missing_list = sorted(list(missing))[:10]  # Top 10 suggestions, sorted
        return f"Consider adding these keywords to your resume: {', '.join(missing_list)}"
    return "Excellent! Your resume contains all the key skills from the job description."

# Function to provide score interpretation
def get_score_feedback(score):
    """
    Provides feedback based on the ATS score.
    """
    if score >= 80:
        return "Excellent match! Your resume aligns very well with this job."
    elif score >= 65:
        return "Good match! Minor improvements could boost your score."
    elif score >= 50:
        return "Moderate match. Review suggestions to improve alignment."
    elif score >= 35:
        return "Below average match. Consider significant improvements."
    else:
        return "Low match. Major revisions recommended to align with job requirements."

# Function to visualize the score
def visualize_score(score):
    """
    Creates a visual representation of the ATS score.
    """
    # Color coding based on score ranges
    if score >= 80:
        color = 'green'
        performance = 'Excellent'
    elif score >= 65:
        color = 'lightgreen'
        performance = 'Good'
    elif score >= 50:
        color = 'orange'
        performance = 'Moderate'
    elif score >= 35:
        color = 'red'
        performance = 'Below Average'
    else:
        color = 'darkred'
        performance = 'Low'
    
    plt.figure(figsize=(10, 6))
    bars = plt.bar(['ATS Match Score'], [score], color=color, alpha=0.8, width=0.6)
    
    plt.ylim(0, 100)
    plt.ylabel('Match Percentage (%)', fontsize=12)
    plt.title('Resume ATS Match Analysis', fontsize=14, fontweight='bold')
    
    # Add score text on the bar
    plt.text(0, score + 2, f'{score:.1f}%\n({performance})', 
             ha='center', va='bottom', fontsize=14, fontweight='bold')
    
    # Add reference lines
    plt.axhline(y=80, color='green', linestyle='--', alpha=0.3, label='Excellent (80%+)')
    plt.axhline(y=65, color='lightgreen', linestyle='--', alpha=0.3, label='Good (65%+)')
    plt.axhline(y=50, color='orange', linestyle='--', alpha=0.3, label='Moderate (50%+)')
    plt.axhline(y=35, color='red', linestyle='--', alpha=0.3, label='Below Average (35%+)')
    
    plt.legend(loc='upper right')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

print("All functions defined successfully.")

In [ ]:
# Step 3: Run the ATS Checker

def run_ats_checker():
    """
    Main function to run the ATS checker.
    """
    clear_output(wait=True)
    
    print("ATS Resume Checker")
    print("=" * 50)
    print("This tool analyzes how well your resume matches a job description.")
    print("Supported formats: PDF, DOC, DOCX")
    print("\nStep 1: Upload your resume file")
    print("Click 'Choose Files' and select your resume...")
    
    # Upload resume file
    uploaded_files = files.upload()
    
    if not uploaded_files:
        print("No file was uploaded. Please try again.")
        return
    
    file_name = list(uploaded_files.keys())[0]
    print(f"Successfully uploaded: {file_name}")
    
    # Extract text from resume
    try:
        resume_text = extract_resume_text(file_name)
        print(f"Extracted {len(resume_text)} characters from your resume.")
        
        if len(resume_text) < 100:
            print("Warning: Very short resume text extracted. Please ensure your PDF/DOC is readable.")
            
    except Exception as e:
        print(f"Error reading file: {e}")
        print("Please ensure you uploaded a valid PDF or DOC file.")
        if os.path.exists(file_name):
            os.remove(file_name)
        return
    
    print("\n" + "="*50)
    print("Step 2: Paste the job description below and press Enter:")
    print("(Paste the complete job posting including requirements and qualifications)")
    
    job_desc = input("\nJob Description: ")
    
    if not job_desc.strip():
        print("Job description cannot be empty. Please try again.")
        if os.path.exists(file_name):
            os.remove(file_name)
        return
    
    if len(job_desc.strip()) < 50:
        print("Warning: Job description seems very short. Consider pasting the complete job posting for better analysis.")
    
    print("\nAnalyzing your resume against the job description...")
    
    try:
        # Extract keywords and calculate score
        resume_keywords = extract_keywords(resume_text)
        job_keywords = extract_keywords(job_desc)
        ats_score = calculate_ats_score(resume_text, job_desc)
        
        # Display comprehensive results
        print("\n" + "="*60)
        print("ATS MATCH ANALYSIS RESULTS")
        print("="*60)
        
        print(f"\nATS Match Score: {ats_score:.2f}%")
        print(f"Score Category: {get_score_feedback(ats_score)}")
        
        print(f"\nKeyword Analysis:")
        print(f"- Resume keywords found: {len(resume_keywords)}")
        print(f"- Job description keywords: {len(job_keywords)}")
        print(f"- Matching keywords: {len(resume_keywords & job_keywords)}")
        
        matching_keywords = resume_keywords & job_keywords
        if matching_keywords:
            print(f"\nMatching skills found: {', '.join(sorted(list(matching_keywords))[:15])}")
        
        print(f"\nImprovement Suggestions:")
        print(get_suggestions(resume_keywords, job_keywords))
        
        # Show visualization
        visualize_score(ats_score)
        
        # Provide additional recommendations
        print("\nAdditional Tips:")
        if ats_score < 65:
            print("- Use exact keywords from the job description in your resume")
            print("- Include relevant skills in your skills section")
            print("- Tailor your experience descriptions to match job requirements")
            print("- Use industry-standard terminology")
        
        print("- Ensure your resume is in a simple, ATS-friendly format")
        print("- Use standard section headings (Experience, Skills, Education)")
        print("- Avoid images, tables, and complex formatting")
        
    except Exception as e:
        print(f"Error during analysis: {e}")
    
    finally:
        # Clean up uploaded file
        if os.path.exists(file_name):
            os.remove(file_name)
            print(f"\nTemporary file '{file_name}' removed.")
    
    print("\n" + "="*60)
    print("Analysis Complete!")
    print("="*60)

print("ATS Checker ready to use.")

In [ ]:
# Step 3: Start the ATS Analysis

# Run this cell to start the ATS checking process
run_ats_checker()

In [ ]:
# Optional: Test with Sample Data

# Uncomment and run this cell to test the functions with sample data

def test_sample_data():
    sample_resume = """
    John Smith
    Software Engineer
    
    Professional Experience:
    Software Developer at Tech Corp (2020-2023)
    - Developed web applications using Python and Django
    - Implemented machine learning models for data analysis
    - Managed PostgreSQL databases and wrote complex SQL queries
    - Collaborated with cross-functional teams using Agile methodology
    - Used Git for version control and Jenkins for CI/CD
    
    Technical Skills:
    Python, JavaScript, SQL, Django, React, PostgreSQL, Git, AWS, Docker
    Machine Learning, Data Analysis, Agile, Scrum
    
    Education:
    Bachelor of Science in Computer Science
    """
    
    sample_job = """
    Senior Software Engineer Position
    
    Requirements:
    - 3+ years of Python development experience
    - Strong knowledge of machine learning and data analysis
    - Experience with SQL databases (PostgreSQL preferred)
    - Frontend development with React.js
    - Cloud services experience (AWS, Docker)
    - Knowledge of Agile development practices
    - Experience with version control (Git)
    - Strong problem-solving and communication skills
    
    Preferred Qualifications:
    - Experience with Kubernetes
    - Knowledge of microservices architecture
    - Bachelor's degree in Computer Science or related field
    """
    
    print("Testing with sample data:")
    print("=" * 40)
    
    resume_kw = extract_keywords(sample_resume)
    job_kw = extract_keywords(sample_job)
    score = calculate_ats_score(sample_resume, sample_job)
    
    print(f"Sample ATS Score: {score:.2f}%")
    print(f"Resume Keywords ({len(resume_kw)}): {sorted(list(resume_kw))[:15]}")
    print(f"Job Keywords ({len(job_kw)}): {sorted(list(job_kw))[:15]}")
    print(f"\n{get_suggestions(resume_kw, job_kw)}")
    
    visualize_score(score)

# Uncomment the line below to run the test
# test_sample_data()